# Datamodel and Database module

The TIdatabase module encapsulates all loading, storing and joining of the student, college and applications dataframes.

The module will be imported at the beginning of every iPython Notebook.

In [10]:
import TIdatabase as ti
%matplotlib inline 
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import sklearn
import statsmodels.api as sm
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
from matplotlib import rcParams

## The Datamodel
This [Google Docs](https://docs.google.com/spreadsheets/d/1dm73Vmov8bhNoVRUtyg6TU-IgE7DPDVlukMkvnaCqAg/edit#gid=0&vpid=A1) contains a list of what we believe are the important factors in the college decision. This is list of course does not include things as recommendation letters as it is impossible to get data for this or to quantifiy it. The doc also includes the column names that each feature has our dataframes. We distinguish 3 dataframes:
- A students dataframe contains all academic and personal data of a particular student (scores, gender, etc)
- A college dataframe contains all information of a university (acceptance rate, public/private, etc)
- An applications dataframe contains application-specific data for a particular student in a particular university, for example and most importantly, the result of the decision procedure

## Generating Mock Data

The module has the functionality to fill the dataframes with mock data, which is useful to start writing  classification code before we finish scraping the actual data.

In [19]:
students = ti.Student()
# populate students with random values
students.fillRandom(10)
students.df 

ValueError: cannot set a row with mismatched columns

#### Simulating Missing Data

You can also simulate NaNs in the mockup. `fillRandom` takes a second optional parameter that is the percentage of NaNs to generate.

In [4]:
students.fillRandom(10, 0.25) # 25% of values will be NaN
students.df

,studentID,classrank,admissionstest,AP,averageAP,SATsubject,GPA,program,schooltype,intendedgradyear,...,female,MinorityGender,MinorityRace,outofstate,international,firstinfamily,alumni,sports,artist,workexp
0,M8L3JM3FBR,0.291678,NaN,0.826825,0.182438,NaN,0.806634,3,NaN,2015,...,0,0,0,1,NaN,1,1,1,1,NaN
1,XQ4UASSSII,0.941785,NaN,NaN,0.533843,NaN,0.259089,NaN,5,2015,...,1,1,1,0,0,0,0,NaN,0,0
2,GWBNEPEY5A,NaN,0.100136,0.647094,0.326618,0.414931,NaN,4,3,2012,...,NaN,1,0,NaN,NaN,1,0,NaN,0,1
3,U91CBM2LJK,0.457057,0.497459,0.843014,NaN,0.720711,0.972963,NaN,5,2020,...,1,1,NaN,1,NaN,0,NaN,0,NaN,1
4,PVJ2YE8FUG,NaN,0.951552,0.658282,0.309046,0.906523,NaN,2,NaN,2016,...,1,0,1,0,1,NaN,1,0,NaN,1
5,TO12VLLKSS,0.696385,0.018087,NaN,NaN,0.059516,0.998190,2,5,NaN,...,0,NaN,NaN,0,1,NaN,0,0,0,1
6,Q8K24POD33,0.310120,0.300553,0.447221,0.286160,0.866734,0.992694,5,1,NaN,...,1,0,NaN,0,NaN,NaN,0,NaN,1,NaN
7,ZHLJGA6TLK,0.536849,0.779149,NaN,0.201465,0.729125,0.948816,NaN,1,2010,...,1,0,0,NaN,1,0,0,NaN,0,1
8,G1I5OF63NC,NaN,0.782187,0.746178,NaN,0.494644,NaN,NaN,2,2012,...,1,NaN,1,1,0,0,NaN,0,1,1
9,ZAYLZVBHKJ,0.888424,0.165326,0.924324,0.634310,0.146511,0.790944,5,4,2019,...,1,0,NaN,1,0,NaN,0,NaN,NaN,1


Since we have a fixed list of only 25 colleges, the college infromation is not scraped.  The list of colleges is populated from a CSV stored in the same directory as this notebook. It can be edited using your favorite CSV editor, such as Excel. When you create a new instance of College, the values will be read in from the CSV. 

In [5]:
# populate with list of known colleges
colleges = ti.College()
colleges.df

,collegeID,name,acceptrate,size,public,finAidPct,instatePct
0,Princeton,Princeton,7.40%,5142,-1,0.00,0
1,Harvard,Harvard,6.00%,19929,-1,0.75,0
2,Yale,Yale,6.30%,12336,-1,0.00,0
3,Columbia,Columbia,7.00%,24221,-1,0.00,0
4,Stanford,Stanford,5.10%,16795,-1,0.00,0
5,UChicago,UChicago,8.80%,12558,-1,0.00,0
6,MIT,MIT,7.90%,11319,-1,0.00,0
7,Duke,Duke,11.40%,15856,-1,0.00,0
8,UPenn,UPenn,10.40%,21296,-1,0.00,0
9,CalTech,CalTech,8.80%,2209,-1,0.00,0


The table of application forms combines a student with a university and carries the information of specific applications. `acceptStatus` is our predictor. `acceptProb` is our $\hat{Y}$ probability.

In [6]:
applForm = ti.ApplForm()
applForm.fillRandom(30)
applForm.df

,studentID,collegeID,earlyAppl,visited,acceptStatus,acceptProb
0,ZHLJGA6TLK,NotreDame,1,1,0,0.227728
1,Q8K24POD33,CalTech,1,1,0,0.194906
2,M8L3JM3FBR,NotreDame,1,1,1,0.722759
3,TO12VLLKSS,Northwestern,1,0,0,0.931419
4,M8L3JM3FBR,UCB,0,1,0,0.590139
5,G1I5OF63NC,MIT,1,1,1,0.189558
6,GWBNEPEY5A,Columbia,1,1,0,0.825882
7,G1I5OF63NC,UChicago,0,0,0,0.642077
8,ZHLJGA6TLK,Duke,1,0,0,0.314230
9,Q8K24POD33,NotreDame,0,0,0,0.058432


To combine the student and application forms tables, we use Pandas merge capability, which will match rows by identical column names, which is studentID in this case:

In [27]:
applications = pd.merge(students.df,applForm.df)
applications

,studentID,classrank,admissionstest,AP,averageAP,SATsubject,GPA,program,schooltype,intendedgradyear,...,firstinfamily,alumni,sports,artist,workexp,collegeID,earlyAppl,visited,acceptStatus,acceptProb
0,RPGQH572X2,NaN,0.98,NaN,NaN,NaN,4.5,NaN,-1,2020,...,NaN,NaN,NaN,NaN,NaN,Harvard,0,0,0,NaN
1,QNNFGQA7TP,NaN,0.98,NaN,NaN,NaN,4.5,NaN,-1,2019,...,NaN,NaN,NaN,NaN,NaN,Yale,0,0,0,NaN
2,Q1LJY003VB,NaN,0.65,NaN,NaN,NaN,2.2,NaN,1,2019,...,NaN,NaN,NaN,NaN,NaN,Columbia,1,1,1,NaN


Now the `applications` Pandas DataFrame is ready to use for either regression (by overwriting the acceptProb column) or building the public facing web site.

## Saving Scraped Data

### Part 1 - The Student Data

First, let's start fresh and delete the previously created objects. This is only necessary because this sample script is running within Jupyter where all variables are global to the page. In a separate Python file run from the command line, this step can be skipped. Creating a new instance would not work as behind the scenes, there is sharing of
DataFrames between objects.

In [8]:
if ('students' in locals()): 
    students.cleanup()
    del students
if ('applications' in locals()): del applications
if ('applForm' in locals()): del applForm


Let's create a new students instance. It will be an empty Pandas dataframe with the correct columns

In [9]:
students = ti.Student()
students.df

,studentID,classrank,admissionstest,AP,averageAP,SATsubject,GPA,program,schooltype,intendedgradyear,...,female,MinorityGender,MinorityRace,outofstate,international,firstinfamily,alumni,sports,artist,workexp


Populate a dictionary with the values that the scraper has for a given row. Make sure the keys match up with the column names as only the matching columns will be saved. There is no need to add the studentID key. A unique value will be generated automatically and returned from the insert. This will be in the same order as the provided rows. Saving the generated student IDs will be helpful later when populating the applForm foreign key.

In [10]:

# Example: international male who scored in 98th percentile in ACT/SAT, went to a public school and is applying for
# Class of 2020

newrow = {'admissionstest': 0.98,
         'GPA': 4.5,
         'female' : -1,
         'international': 1,
         'schooltype': -1,
         'intendedgradyear':2020}

newsinglestudentID = students.insert(newrow)
print "New studentID:",newsinglestudentID
students.df

New studentID: ['RPGQH572X2']


,studentID,classrank,admissionstest,AP,averageAP,SATsubject,GPA,program,schooltype,intendedgradyear,...,female,MinorityGender,MinorityRace,outofstate,international,firstinfamily,alumni,sports,artist,workexp
0,RPGQH572X2,NaN,0.98,NaN,NaN,NaN,4.5,NaN,-1,2020,...,-1,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN


It is more efficient if multiple rows are added in one step. In this case, create a list of dictionaries and just
use the same method. Here, two new rows are added to the DataFrame in one step.

In [11]:
rows = []
a = {'schooltype': -1, 'admissionstest': 0.98, 'GPA': 4.5, 'female': 1, 'intendedgradyear': 2019, 'international': 0}
rows.append(a)
a = {'schooltype': 1, 'admissionstest': 0.65, 'GPA': 2.2, 'female': -1, 'intendedgradyear': 2019, 'international': 0}
rows.append(a)
newmanystudentIDs = students.insert(rows)
print "New studentIDs:",newmanystudentIDs
students.df

New studentIDs: ['QNNFGQA7TP', 'Q1LJY003VB']


,studentID,classrank,admissionstest,AP,averageAP,SATsubject,GPA,program,schooltype,intendedgradyear,...,female,MinorityGender,MinorityRace,outofstate,international,firstinfamily,alumni,sports,artist,workexp
0,RPGQH572X2,NaN,0.98,NaN,NaN,NaN,4.5,NaN,-1,2020,...,-1,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
0,QNNFGQA7TP,NaN,0.98,NaN,NaN,NaN,4.5,NaN,-1,2019,...,1,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
1,Q1LJY003VB,NaN,0.65,NaN,NaN,NaN,2.2,NaN,1,2019,...,-1,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN


Now we are ready to save. The data is saved in CSV format for ease of interpretability.

In [12]:
students.save("mydata.csv")

Let's delete all the data and check that we can read it back successfully.

In [13]:
if ('students' in locals()): 
    students.cleanup()
    del students
if ('applications' in locals()): del applications
if ('applForm' in locals()): del applForm



In [14]:
students = ti.Student()
students.read("mydata.csv")
students.df

,studentID,classrank,admissionstest,AP,averageAP,SATsubject,GPA,program,schooltype,intendedgradyear,...,female,MinorityGender,MinorityRace,outofstate,international,firstinfamily,alumni,sports,artist,workexp
0,RPGQH572X2,NaN,0.98,NaN,NaN,NaN,4.5,NaN,-1,2020,...,-1,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
0,QNNFGQA7TP,NaN,0.98,NaN,NaN,NaN,4.5,NaN,-1,2019,...,1,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
1,Q1LJY003VB,NaN,0.65,NaN,NaN,NaN,2.2,NaN,1,2019,...,-1,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN


Et, voilà, the data is back.

### Part 2 - The Application Data

This is pretty much the same, **except** for two important differences:

* The studentID and collegeID must both be populated and exist in the respective DataFrames
* The columns are a little different. Normally acceptProb would not be populated from the scraper but could be used to store prediction runs. 

In [15]:
# we already wiped out applForm above
applForm = ti.ApplForm()
applForm.df

,studentID,collegeID,earlyAppl,visited,acceptStatus,acceptProb


In [16]:
# either pick one from the students.df DataFrame, like this:
#--- studentID = students.df.iloc[1].studentID
# or use the studentID from the insert in the students DataFrame
studentID = newsinglestudentID[0]
collegeID = colleges.df.iloc[1].collegeID

In [17]:
print studentID, collegeID

RPGQH572X2 Harvard


In [18]:
newrow = {'studentID': studentID,
         'collegeID': collegeID,
         'earlyAppl' : 0,
         'visited': 0,
         'acceptStatus': 0}

applForm.insert(newrow)
applForm.df

,studentID,collegeID,earlyAppl,visited,acceptStatus,acceptProb
0,RPGQH572X2,Harvard,0,0,0,NaN


Now inserting multiple rows

In [19]:
rows = []
# either pick one randomly:
#--- studentID = students.df.iloc[2].studentID
# or use the list from the insert into the students DataFrame
# iterate over the list of newstudentIDs if necessary
studentID = newmanystudentIDs[0]
collegeID = colleges.df.iloc[2].collegeID
newrow = {'studentID': studentID,
         'collegeID': collegeID,
         'earlyAppl' : 0,
         'visited': 0,
         'acceptStatus': 0}
rows.append(newrow)
# note: same student, new school to apply to
studentID = students.df.iloc[2].studentID
collegeID = colleges.df.iloc[3].collegeID
newrow = {'studentID': studentID,
         'collegeID': collegeID,
         'earlyAppl' : 1,
         'visited': 1,
         'acceptStatus': 1}
rows.append(newrow)
applForm.insert(rows)
applForm.df

,studentID,collegeID,earlyAppl,visited,acceptStatus,acceptProb
0,RPGQH572X2,Harvard,0,0,0,NaN
0,QNNFGQA7TP,Yale,0,0,0,NaN
1,Q1LJY003VB,Columbia,1,1,1,NaN


Let's save it

In [20]:
applForm.save("applform1.csv")

Then delete the local variable

In [21]:
if ('applForm' in locals()): del applForm

Then read it back

In [22]:
applForm = ti.ApplForm()
applForm.read("applform1.csv")
applForm.df


,studentID,collegeID,earlyAppl,visited,acceptStatus,acceptProb
0,RPGQH572X2,Harvard,0,0,0,NaN
0,QNNFGQA7TP,Yale,0,0,0,NaN
1,Q1LJY003VB,Columbia,1,1,1,NaN


And now let's check that the merge still works

In [23]:
applications = pd.merge(students.df,applForm.df)
applications

,studentID,classrank,admissionstest,AP,averageAP,SATsubject,GPA,program,schooltype,intendedgradyear,...,firstinfamily,alumni,sports,artist,workexp,collegeID,earlyAppl,visited,acceptStatus,acceptProb
0,RPGQH572X2,NaN,0.98,NaN,NaN,NaN,4.5,NaN,-1,2020,...,NaN,NaN,NaN,NaN,NaN,Harvard,0,0,0,NaN
1,QNNFGQA7TP,NaN,0.98,NaN,NaN,NaN,4.5,NaN,-1,2019,...,NaN,NaN,NaN,NaN,NaN,Yale,0,0,0,NaN
2,Q1LJY003VB,NaN,0.65,NaN,NaN,NaN,2.2,NaN,1,2019,...,NaN,NaN,NaN,NaN,NaN,Columbia,1,1,1,NaN


Ok, we are done for today.